# Emission Measure
In this notebook, we'll generate the emission measure slope maps for each of our heating cases and save them as FITS files.

In [1]:
import os
import sys
import glob
import copy

import numpy as np
from scipy.interpolate import splev
import matplotlib
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.coordinates import SkyCoord
import distributed

import synthesizAR
from synthesizAR.instruments import InstrumentSDOAIA
from synthesizAR.analysis import DistributedAIACollection,DistributedAIACube
from synthesizAR.analysis.dem import EMCube

sys.path.append('../scripts/')
from dem import HannahKontarModel, make_slope_map_tpeak

import warnings
warnings.filterwarnings('ignore',category=UserWarning,)

%matplotlib inline

In [2]:
cluster = distributed.LocalCluster(n_workers=32,threads_per_worker=2)
client = distributed.Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:34228 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 32 Cores: 64 Memory: 270.38 GB


First, read in the model and observational data. For now, we'll just take a time-average. It may be better to time-average in chunks though still not completely sure about that...

In [3]:
intensity_file_format = '/storage-home/w/wtb2/data/timelag_synthesis_v2/{}/nei/SDO_AIA/{}/map_t{:06d}.fits'
emslope_file_format = '/storage-home/w/wtb2/projects/synthetic-observables-paper-models/paper/data/{}/em_slope.fits'

Next, set up the temperature bins.

In [4]:
temperature_bin_edges = 10.**np.arange(5.5,7.3,0.1) * u.K
temperature_bin_centers = (temperature_bin_edges[1:] + temperature_bin_edges[:-1])/2.

And calculate the instrument responses.

In [9]:
aia = InstrumentSDOAIA([0,1]*u.s,None)

In [10]:
responses = [splev(temperature_bin_centers.value, c['temperature_response_spline']) for c in aia.channels]

Now, we'll go through each heating case and compute the emission measure distribution.

### High

In [11]:
cube = DistributedAIACollection(*[DistributedAIACube.from_files(
    [intensity_file_format.format('high_frequency', c['name'], i) for i in range(500,2500)]) for c in aia.channels])

In [16]:
maps = [cube[c['name']].average() for c in aia.channels]

In [17]:
hk_model = HannahKontarModel(maps, temperature_bin_edges, responses,
                             dem_path='/storage-home/w/wtb2/codes/demreg/idl')

In [18]:
em_high = hk_model.fit(
    alpha=1,
    increase_alpha=1.1,
    verbose=False,
    n_sample=len(cube[0].maps)
)

In [37]:
slope_map = make_slope_map_tpeak(
    em_high,
    Tmin=10**(6)*u.K,
    safety=10**(0.3),
    rsquared_tolerance=0.9
)
slope_map.save(emslope_file_format.format('high_frequency'),overwrite=True)

## Intermediate 

In [24]:
cube = DistributedAIACollection(*[DistributedAIACube.from_files(
    [intensity_file_format.format('intermediate_frequency', c['name'], i) for i in range(500,2500)]) for c in aia.channels])

In [25]:
maps = [cube[c['name']].average() for c in aia.channels]

In [26]:
hk_model = HannahKontarModel(
    maps,
    temperature_bin_edges,
    responses,
    dem_path='/storage-home/w/wtb2/codes/demreg/idl'
)

In [27]:
em_intermediate = hk_model.fit(
    alpha=1,
    increase_alpha=1.1,
    verbose=False,
    n_sample=len(cube[0].maps)
)

In [39]:
slope_map = make_slope_map_tpeak(
    em_intermediate,
    Tmin=10**(6)*u.K,
    safety=10**(0.3),
    rsquared_tolerance=0.9
)
slope_map.save(emslope_file_format.format('intermediate_frequency'),overwrite=True)

/storage-home/w/wtb2/anaconda3/envs/synthetic-observables/lib/python3.6/site-packages/synthesizAR-0.1.dev856-py3.6.egg/synthesizAR/analysis/dem.py:125: RuntimeWarning: divide by zero encountered in log10
  em_fit = np.log10(data.value.reshape((np.prod(data.shape[:2]),) + data.shape[2:]).T)
/storage-home/w/wtb2/anaconda3/envs/synthetic-observables/lib/python3.6/site-packages/synthesizAR-0.1.dev856-py3.6.egg/synthesizAR/analysis/dem.py:133: RuntimeWarning: divide by zero encountered in true_divide
  rsquared = 1. - rss/rss_flat
/storage-home/w/wtb2/anaconda3/envs/synthetic-observables/lib/python3.6/site-packages/synthesizAR-0.1.dev856-py3.6.egg/synthesizAR/analysis/dem.py:133: RuntimeWarning: invalid value encountered in true_divide
  rsquared = 1. - rss/rss_flat
/storage-home/w/wtb2/anaconda3/envs/synthetic-observables/lib/python3.6/site-packages/synthesizAR-0.1.dev856-py3.6.egg/synthesizAR/analysis/dem.py:134: RuntimeWarning: invalid value encountered in less
  rsquared_mask = rsquared

## Low

In [30]:
cube = DistributedAIACollection(*[DistributedAIACube.from_files(
    [intensity_file_format.format('low_frequency', c['name'], i) for i in range(500,2500)]) for c in aia.channels])

In [31]:
maps = [cube[c['name']].average() for c in aia.channels]

In [32]:
hk_model = HannahKontarModel(
    maps,
    temperature_bin_edges,
    responses,
    dem_path='/storage-home/w/wtb2/codes/demreg/idl'
)

In [33]:
em_low = hk_model.fit(
    alpha=1,
    increase_alpha=1.1,
    verbose=False,
    n_sample=len(cube[0].maps)
)

In [40]:
slope_map = make_slope_map_tpeak(
    em_low,
    Tmin=10**(6)*u.K,
    safety=10**(0.3),
    rsquared_tolerance=0.9
)
slope_map.save(emslope_file_format.format('low_frequency'),overwrite=True)